# IMPORTANT: 
This notebook's intended purpose is to be used in google collab to train and download the model. For usage, this notebook should be downloaded, uploaded to the drive, and only then used. The config files included in the darknet folder are also to be used in google drive and have no use in this repository other than display purposes.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Setup:

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (build)
!make

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# copy and unzip training data
!cp /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/data/obj.zip ../
!unzip ../obj.zip -d data/

In [ ]:
# use generate_train script to format our training files correctly
!cp /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/generate_train.py ./
!python generate_train.py

In [ ]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
# upload the custom config file, obj names and obj data
!cp /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/yolov3_custom.cfg ./cfg
!cp /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/obj.names.txt ./data
!cp /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/obj.data.txt  ./data

In [ ]:
# This stops 'Run all' at this cell by causing an error
assert False

# Training:

In [ ]:
# train custom detector
!./darknet detector train /content/darknet/data/obj.data.txt /content/darknet/cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

# use to resume training from previously trained weights
#!./darknet detector train /content/darknet/data/obj.data.txt /content/darknet/cfg/yolov3_custom.cfg /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/backup/yolov3_custom_last.weights -dont_show 

In [ ]:
# show the training chart, doesn't work if the machine stops in the middle
imShow('chart.png')

# Using Darknet for Predictions:

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..


In [ ]:
# predicting
!./darknet detector test data/obj.data.txt cfg/yolov3_custom.cfg /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/backup/yolov3_custom_last.weights /content/drive/MyDrive/DeepLearning/FinalProject/images/img2.jpg -thresh 0.3
imShow('predictions.jpg')

In [ ]:
# writing the outputs to a text file
!./darknet detector test data/obj.data.txt cfg/yolov3_custom.cfg /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/backup/yolov3_custom_last.weights -ext_output /content/drive/MyDrive/DeepLearning/FinalProject/images/img2.jpg -thresh 0.3 -save_labels > results.txt

Credits to "The AI Guy" on youtube, who created the original notebook that I modified. Thanks!

[The AI Guy](https://www.youtube.com/channel/UCrydcKaojc44XnuXrfhlV8Q)

# Converting the weights file into a tensorflow model:

In [ ]:
# cloning the repository used
!git clone https://github.com/hunglc007/tensorflow-yolov4-tflite

In [ ]:
%cd tensorflow-yolov4-tflite
!cp /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/coco.names ./data/classes # need to create a coco.names file with the classes

In [ ]:
!python save_model.py --weights /content/drive/MyDrive/DeepLearning/FinalProject/Test2-only_notes/yolov3.weights --output ./checkpoints/yolov3-704 --input_size 704 --model yolov3
# Then download manually from .darknet/tensorflow-yolov3-tflite/checkpoints/yolov3-704